In [ ]:
!pip install transformers datasets sacrebleu sentencepiece accelerate evaluate
!pip install -U transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from datasets import Dataset
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate
import torch

df = pd.read_csv("/content/gujlish_data.csv")

df.head()


# Convert pandas dataframe -> Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Train/validation split (90-10%)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_data = dataset["train"]
val_data = dataset["test"]

# nllb-200 model

In [3]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import pandas as pd
import evaluate

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
HF_TOKEN=""
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=HF_TOKEN)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=HF_TOKEN)

src_lang = "eng_Latn"   
tgt_lang = "eng_Latn"   

tokenizer.src_lang = src_lang
tokenizer.tgt_lang = tgt_lang

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:

max_input_length = 64
max_target_length = 64

def preprocess(batch):
    inputs = [ex for ex in batch["guj"]]
    targets = [ex for ex in batch["src"]]

    model_inputs = tokenizer(
        inputs, max_length=max_input_length, truncation=True, padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, max_length=max_target_length, truncation=True, padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_data.map(preprocess, batched=True, remove_columns=train_data.column_names)
val_dataset = val_data.map(preprocess, batched=True, remove_columns=val_data.column_names)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:

from transformers import DataCollatorForSeq2Seq
import sacrebleu

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 with pad_token_id
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # SacreBLEU expects list of predictions and list of list of references
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])
    return {"bleu": bleu.score}


In [ ]:

training_args = Seq2SeqTrainingArguments(
    output_dir="./nllb_guj_eng_translation",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    save_total_limit=2,
    report_to="none"   # disable wandb
)

In [ ]:

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-431645207.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:

trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,0.396300,0.370938,39.889926
2,0.293500,0.341976,41.280986
3,0.270900,0.335734,41.647326
4,0.232100,0.335938,41.446267


Epoch,Training Loss,Validation Loss,Bleu
1,0.396300,0.370938,39.889926
2,0.293500,0.341976,41.280986
3,0.270900,0.335734,41.647326
4,0.232100,0.335938,41.446267
5,0.198800,0.340781,41.500789


TrainOutput(global_step=15000, training_loss=0.3706493332544963, metrics={'train_runtime': 9434.2277, 'train_samples_per_second': 12.72, 'train_steps_per_second': 1.59, 'total_flos': 1.625328451584e+16, 'train_loss': 0.3706493332544963, 'epoch': 5.0})

In [ ]:

trainer.save_model("./nllb_guj_eng_translation")
tokenizer.save_pretrained("./nllb_guj_eng_translation")

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./nllb_eng_guj_translation/tokenizer_config.json',
 './nllb_eng_guj_translation/special_tokens_map.json',
 './nllb_eng_guj_translation/sentencepiece.bpe.model',
 './nllb_eng_guj_translation/added_tokens.json',
 './nllb_eng_guj_translation/tokenizer.json')

In [ ]:

model = AutoModelForSeq2SeqLM.from_pretrained("./nllb_guj_eng_translation")
tokenizer = AutoTokenizer.from_pretrained("./nllb_guj_eng_translation")

def translate_gujlish(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model.generate(**inputs, max_length=64, num_beams=5)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test
examples = [
    "Robert Jones, umar 43, Tuesday na Calderdale Magistrates Court ma present hato.",
    "Shu tame mari saathe party ma javu chho?",
    "Jyaare alarm ring kare chhe, tyare ketlu time lage chhe.",
]
for ex in examples:
    print(f"\nGujlish: {ex}")
    print(f"English: {translate_gujlish(ex)}")


Gujlish: Robert Jones, aged 43, was present at Calderdale Magistrates Court on Tuesday.
English: Robert Jones, umar 43, Tuesday na Calderdale Magistrates Court ma present hato.

Gujlish: Do you want to go to a party with me?
English: Shu tame mari saathe party ma javu chho?

Gujlish: When the alarm rings, how long does it take.
English: Jyaare alarm ring kare chhe, tyare ketlu time lage chhe.
